# Feature Engineering

In [1]:
import pandas as pd
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq

from tqdm import tqdm

import time
import gc

pd.set_option('display.max_columns', None)

## Daten laden

In [2]:
IS_TRAINING = False
FILE = '../../data/processed/v2/validation_series_split_normalized.parquet'
series = pd.read_parquet(FILE, columns=['series_id'])

## Neue Features erstellen

In [3]:
folder = "train/" if IS_TRAINING else "validation/"
file_path_prefix = "../../data/processed/v2/" + folder

def save_chunk(series_id, chunck):
    chunck.to_parquet(file_path_prefix + series_id + ".parquet")

In [4]:
def make_features_chunck(series_id):
    df = pd.read_parquet(FILE, filters=[('series_id','=',series_id)])
          
    for i in range(1, 25):
        df[f"anglez_past-lag_{i}"] = df["anglez"].shift(i).bfill()
        df[f"enmo_past-lag_{i}"] = df["enmo"].shift(i).bfill()

    for i in range(-1, -25, -1):
        df[f"anglez_future-lag_{i}"] = df["anglez"].shift(i).ffill()
        df[f"enmo_future-lag_{i}"] = df["enmo"].shift(i).ffill()
    
    return df

In [5]:
def make_features(series):
    for series_id in tqdm(series.series_id.unique()):
        chunck = make_features_chunck(series_id)
        save_chunk(series_id, chunck)
        
        del chunck
        gc.collect()

In [6]:
start_time = time.time()
make_features(series)
print(f'Feature Engineering took {time.time() - start_time} seconds')

  0%|                                                                                           | 0/54 [00:00<?, ?it/s]C:\Users\timon\AppData\Local\Temp\ipykernel_8516\1279877347.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"enmo_future-lag_{i}"] = df["enmo"].shift(i).ffill()
  2%|█▌                                                                                 | 1/54 [00:06<05:21,  6.07s/it]C:\Users\timon\AppData\Local\Temp\ipykernel_8516\1279877347.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"enmo_fut

 30%|████████████████████████▎                                                         | 16/54 [01:09<02:44,  4.32s/it]C:\Users\timon\AppData\Local\Temp\ipykernel_8516\1279877347.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"enmo_future-lag_{i}"] = df["enmo"].shift(i).ffill()
 31%|█████████████████████████▊                                                        | 17/54 [01:13<02:32,  4.13s/it]C:\Users\timon\AppData\Local\Temp\ipykernel_8516\1279877347.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"enmo_fut

 59%|████████████████████████████████████████████████▌                                 | 32/54 [02:18<01:40,  4.55s/it]C:\Users\timon\AppData\Local\Temp\ipykernel_8516\1279877347.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"enmo_future-lag_{i}"] = df["enmo"].shift(i).ffill()
 61%|██████████████████████████████████████████████████                                | 33/54 [02:20<01:24,  4.02s/it]C:\Users\timon\AppData\Local\Temp\ipykernel_8516\1279877347.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"enmo_fut

 89%|████████████████████████████████████████████████████████████████████████▉         | 48/54 [03:26<00:27,  4.54s/it]C:\Users\timon\AppData\Local\Temp\ipykernel_8516\1279877347.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"enmo_future-lag_{i}"] = df["enmo"].shift(i).ffill()
 91%|██████████████████████████████████████████████████████████████████████████▍       | 49/54 [03:30<00:21,  4.31s/it]C:\Users\timon\AppData\Local\Temp\ipykernel_8516\1279877347.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"enmo_fut

Feature Engineering took 236.292578458786 seconds


## Overview speichern

In [7]:
overview = pd.read_parquet(FILE, columns=['series_id', 'step'])

In [8]:
overview.to_parquet(file_path_prefix + "overview.parquet")